In [ ]:
import glob                 #To get files from folder
import pandas as pd         #To read those files
import numpy as np          #To do mathematical operation
import os                   #Files manupulation 

from sklearn.naive_bayes import MultinomialNB                       #Model import from sklearn
from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.model_selection import train_test_split                 
from sklearn.metrics import accuracy_score                           #How the classifire works/ How accruate it is!
import time                                                          #How much time it take to run
import re                                                            #Replacement
from sklearn.pipeline import Pipeline                                
from sklearn.feature_extraction.text import TfidfTransformer         
from sklearn.metrics import confusion_matrix                         #Model description with color 
import joblib                                                        


In [ ]:
all_file = glob.glob("../songData/*.xlsx")
len(all_file)

In [ ]:
all_file

g = all_file[2]
g.split('\\')[1][:-5]

In [ ]:
names = []
count = []
clusters = []
data = []

for each in all_file:
    temp = pd.read_excel(each,engine='openpyxl')
    
    
    count.append(len(temp))
    names.append(each.split('/')[1][:-5])
    clusters.append(temp['songType'])
    
    
    data.append(temp)
    
cluster_ids =clusters    

In [ ]:
data = pd.concat(data)
data = data.sample(frac =1)

data

In [310]:
data['lyrics']

656    আমার বেলা যে যায় সাঁঝ-বেলাতে তোমার সুরে সুরে স...
318    চল না হারিয়ে যাই কোন দূর অজানায় যেখানে ফেলে ...
142    এ কি অপরুপ রুপে মা তোমায় হেরিনু পল্লী-জননী। ফু...
498    ধ্বংসের সীমানায়, স্বপ্নে সাজানো আঙিনায় তুমি ...
5      বঁধুয়া নিদ নাহি আঁখিপাতে। আমিও একাকী, তুমিও এক...
                             ...                        
115    একলা বসে বাদল-শেষে শুনি কত কী– ‘এবার আমার গেল ...
570    হায়রে কপাল মন্দ চোখ থাকিতে অন্ধ এই জীবন জ্বইল...
457    জীবনানন্দের কবিতা উপমায় একটু মিথ্যে নেই পৃথিবী...
568    ওঠো ওঠো রে– বিফলে প্রভাত বহে যায় যে। মেলো আঁখ...
220    বসন্তপ্রভাতে এক মালতীর ফুল প্রথম মেলিল আঁখি তা...
Name: lyrics, Length: 4105, dtype: object

In [311]:
stopwords = [ 'হাই', 'ফ্রিকুয়েন্সি', 'কিন্তু', 'অপ্রয়োজনীয়', 'শব্দ' ]

In [312]:
data['lyrics'] = data['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [313]:
X_train, X_test, Y_train, Y_test = train_test_split(data['lyrics'],data['songType'],test_size=0.33, random_state=42 )

In [314]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import metrics

total_data_count = len(data)
per_class_counts = []
unique_classes = np.unique(data['songType'].values)
class_count_mean = np.mean(count)

############### classifier setup ################

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([
    
    ('vect', CountVectorizer(stop_words=stopwords,
                            analyzer="word", 
                            lowercase=False,
                            token_pattern="[\S]*",
                            tokenizer=None,
                            ngram_range=(1, 3),
                            preprocessor=None)),                 
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(loss = 'modified_huber', penalty='l2', max_iter=100)),
                                                                        
])
classifier = text_clf_svm.fit(list(X_train), list(Y_train))


cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
score = cross_val_score(text_clf_svm, list(X_train), list(Y_train), cv=cv)

print("Cross Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))


predicted = classifier.predict(X_test)

C:\Users\MANSTEIN\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [''] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Cross Accuracy: 0.53 (+/- 0.03)


In [315]:
joblib.dump(classifier, '../Model/sample_model.pickle')

['../Model/sample_model.pickle']

In [316]:
print("Cross Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
print(metrics.classification_report(Y_test, predicted))

Cross Accuracy: 0.53 (+/- 0.03)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.51      0.37      0.43       344
           2       0.00      0.00      0.00         1
           3       0.37      0.70      0.48       267
           4       0.50      0.25      0.33         4
           5       1.00      0.04      0.08        23
           6       0.45      0.39      0.42        23
           7       0.62      0.33      0.43        73
           8       0.00      0.00      0.00         8
           9       0.00      0.00      0.00        20
          10       0.00      0.00      0.00        16
          11       0.14      0.10      0.11        21
          12       0.12      0.02      0.04        41
          13       0.50      0.45      0.47       220
          14       1.00      0.10      0.17        21
          16       0.00      0.00      0.00         1
          17       0.77      0.92      0.84      

C:\Users\MANSTEIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [317]:
import pandas as pd
confusion_matrix = list(metrics.confusion_matrix(Y_test, predicted))

In [318]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix
df_cm = pd.DataFrame(array, index = [i for i in names],
              columns = [i for i in names])
df_norm_col=(df_cm-df_cm.mean())/df_cm.std()
#df_norm_col=df_cm-df_cm.mean()
plt.figure(figsize = (15,10))
sn.heatmap(df_norm_col, annot=True,cmap='viridis')

ValueError: 21 columns passed, passed data had 20 columns